In [1]:
from google.colab import drive
drive.mount('/content/drive')

#글꼴 설치
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fi
# !rm ~/. cache/matplotlib -rf

Mounted at /content/drive


In [2]:
#라이브러리 임포트
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as ss
import warnings
warnings.filterwarnings(action='ignore')
pd.set_option('display.max_columns', None)

#파일 불러오기
cont_fi = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종프/전처리완료파일/cont_fi.csv')
mgg_fi = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종프/전처리완료파일/mgg_fi.csv')

In [3]:
print(cont_fi.shape)
print(mgg_fi.shape)

(225640, 12)
(1290465, 4)


In [4]:
display(cont_fi)
display(mgg_fi)

,POL_SN,reduce_GBN_CD,reduce_STAT_CD,YM_202006,CT_CHNL_CD,CT_PY_AMT,CT_PY_CYCLE_CD,CT_PY_PD,mean_AMT,month_AMT,z_score,m_z_score
0,37,6,4,1,1,3,1,0,3.0,3.000000,-0.045782,-0.044223
1,43,1,2,1,4,5000,1,0,5000.0,5000.000000,1.329212,1.337227
2,66,3,2,1,4,98,3,1,98.0,8.166667,-0.019641,-0.042795
3,87,7,1,1,1,1300,2,2,1300.0,1300.000000,0.311106,0.314340
4,111,1,1,1,3,60,2,25,60.0,60.000000,-0.030098,-0.028465
...,...,...,...,...,...,...,...,...,...,...,...,...
225635,9997065,1,1,1,1,190,2,20,190.0,190.000000,0.005674,0.007474
225636,9997240,6,4,1,3,24,1,0,24.0,24.000000,-0.040004,-0.038418
225637,9997268,2,1,1,1,31,2,20,65.0,65.000000,-0.028722,-0.027083
225638,9997320,4,2,1,1,81,2,30,81.0,81.000000,-0.024319,-0.022660


,JOIN_SN,POL_SN,MGG_GRP_CD,reduce_GRN_CD
0,499723,3260523,A5,3
1,998541,1272317,A4,2
2,1189610,7519869,A3,1
3,1464515,1941349,A4,2
4,1535736,9540274,A1,1
...,...,...,...,...
1290460,1737843,667896,A1,1
1290461,21056,1660383,A6,3
1290462,1537144,5663552,A3,1
1290463,1608352,9919009,A4,2


# 중복행 제거하면서 백만개 행 사라짐.

In [5]:
mgg_fi.drop_duplicates(subset=['JOIN_SN','POL_SN','reduce_GRN_CD'], keep='first', inplace=True)
mgg_fi.shape

(297096, 4)

In [6]:
mgg_fi.sort_values(by='POL_SN')

,JOIN_SN,POL_SN,MGG_GRP_CD,reduce_GRN_CD
242736,902434,8,A96,4
211131,902434,8,A4,2
144156,902434,8,A1,1
49501,902434,8,A6,3
10606,1667175,111,A5,3
...,...,...,...,...
482904,1101388,9997286,A4,2
157035,545697,9997320,A5,3
350556,545697,9997320,A3,1
232134,545697,9997320,A4,2


In [7]:
print(mgg_fi['JOIN_SN'].nunique(), mgg_fi['POL_SN'].nunique(),cont_fi['POL_SN'].nunique())

38825 110823 225640


# 피벗테이블 만들기

In [8]:
mgg_pivot = pd.pivot_table(mgg_fi, index=['POL_SN',	'JOIN_SN'], \
                           columns = 'reduce_GRN_CD', values = 'MGG_GRP_CD',\
                           aggfunc='count', fill_value = 0)

mgg_pivot

,reduce_GRN_CD,1,2,3,4
POL_SN,JOIN_SN,,,,
8,902434,1,1,1,1
111,1667175,1,1,1,0
179,594950,1,1,1,0
311,1564167,1,1,1,0
478,372209,1,1,1,1
...,...,...,...,...,...
9997129,1406609,1,1,0,1
9997240,846546,1,0,0,0
9997274,1494449,0,1,0,0


In [9]:
mgg_pivot.columns = ['GBN_CD_1', 'GBN_CD_2','GBN_CD_3','GBN_CD_4']
mgg_pivot

,,GBN_CD_1,GBN_CD_2,GBN_CD_3,GBN_CD_4
POL_SN,JOIN_SN,,,,
8,902434,1,1,1,1
111,1667175,1,1,1,0
179,594950,1,1,1,0
311,1564167,1,1,1,0
478,372209,1,1,1,1
...,...,...,...,...,...
9997129,1406609,1,1,0,1
9997240,846546,1,0,0,0
9997274,1494449,0,1,0,0


### 계약정보에만 있는 증권번호

In [10]:
not_mgg = cont_fi[~cont_fi['POL_SN'].isin(mgg_fi['POL_SN'])]
not_mgg

,POL_SN,reduce_GBN_CD,reduce_STAT_CD,YM_202006,CT_CHNL_CD,CT_PY_AMT,CT_PY_CYCLE_CD,CT_PY_PD,mean_AMT,month_AMT,z_score,m_z_score
0,37,6,4,1,1,3,1,0,3.0,3.000000,-0.045782,-0.044223
1,43,1,2,1,4,5000,1,0,5000.0,5000.000000,1.329212,1.337227
2,66,3,2,1,4,98,3,1,98.0,8.166667,-0.019641,-0.042795
3,87,7,1,1,1,1300,2,2,1300.0,1300.000000,0.311106,0.314340
6,231,2,3,1,1,46,2,20,46.0,46.000000,-0.033950,-0.032336
...,...,...,...,...,...,...,...,...,...,...,...,...
225628,9996427,5,2,1,3,30,2,20,30.0,30.000000,-0.038353,-0.036759
225630,9996475,9,2,1,1,120,2,5,120.0,120.000000,-0.013588,-0.011878
225632,9996560,8,2,1,6,35,2,10,35.0,35.000000,-0.036977,-0.035377
225637,9997268,2,1,1,1,31,2,20,65.0,65.000000,-0.028722,-0.027083


# 3&4 테이블 merge
- left_merge_3_4 => 담보없는 계약데이터 포함 (결측치는 0으로 대체)
- inner_merge_3_4 => 두 테이블 공통 데이터만 취함

In [11]:
left_merge_3_4 = pd.merge(cont_fi, mgg_pivot, on='POL_SN', how='left',)
inner_merge_3_4 = pd.merge(cont_fi, mgg_pivot, on='POL_SN', how='inner')

In [12]:
left_merge_3_4 = left_merge_3_4.fillna(0)
display(left_merge_3_4)
display(inner_merge_3_4)

,POL_SN,reduce_GBN_CD,reduce_STAT_CD,YM_202006,CT_CHNL_CD,CT_PY_AMT,CT_PY_CYCLE_CD,CT_PY_PD,mean_AMT,month_AMT,z_score,m_z_score,GBN_CD_1,GBN_CD_2,GBN_CD_3,GBN_CD_4
0,37,6,4,1,1,3,1,0,3.0,3.000000,-0.045782,-0.044223,0.0,0.0,0.0,0.0
1,43,1,2,1,4,5000,1,0,5000.0,5000.000000,1.329212,1.337227,0.0,0.0,0.0,0.0
2,66,3,2,1,4,98,3,1,98.0,8.166667,-0.019641,-0.042795,0.0,0.0,0.0,0.0
3,87,7,1,1,1,1300,2,2,1300.0,1300.000000,0.311106,0.314340,0.0,0.0,0.0,0.0
4,111,1,1,1,3,60,2,25,60.0,60.000000,-0.030098,-0.028465,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225656,9997065,1,1,1,1,190,2,20,190.0,190.000000,0.005674,0.007474,1.0,1.0,1.0,0.0
225657,9997240,6,4,1,3,24,1,0,24.0,24.000000,-0.040004,-0.038418,1.0,0.0,0.0,0.0
225658,9997268,2,1,1,1,31,2,20,65.0,65.000000,-0.028722,-0.027083,0.0,0.0,0.0,0.0
225659,9997320,4,2,1,1,81,2,30,81.0,81.000000,-0.024319,-0.022660,1.0,1.0,1.0,1.0


,POL_SN,reduce_GBN_CD,reduce_STAT_CD,YM_202006,CT_CHNL_CD,CT_PY_AMT,CT_PY_CYCLE_CD,CT_PY_PD,mean_AMT,month_AMT,z_score,m_z_score,GBN_CD_1,GBN_CD_2,GBN_CD_3,GBN_CD_4
0,111,1,1,1,3,60,2,25,60.000000,60.000000,-0.030098,-0.028465,1,1,1,0
1,179,1,1,1,1,330,2,20,330.000000,330.000000,0.044197,0.046178,1,1,1,0
2,311,3,1,1,4,120,3,1,120.000000,10.000000,-0.013588,-0.042288,1,1,1,0
3,478,4,1,1,1,110,2,20,110.000000,110.000000,-0.016339,-0.014643,1,1,1,1
4,722,3,1,1,1,28,2,30,28.000000,28.000000,-0.038903,-0.037312,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110839,9996700,1,1,1,1,47,2,20,45.666667,45.666667,-0.034042,-0.032428,1,1,1,0
110840,9996733,4,1,1,1,14,2,10,14.000000,14.000000,-0.042755,-0.041182,1,1,1,1
110841,9997065,1,1,1,1,190,2,20,190.000000,190.000000,0.005674,0.007474,1,1,1,0
110842,9997240,6,4,1,3,24,1,0,24.000000,24.000000,-0.040004,-0.038418,1,0,0,0
